In [1]:
from os import getcwd, path, environ
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

BASE_PATH = getcwd()
sys.path.append(BASE_PATH)

environ['NUM_WORKERS'] = '0'
# environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
data = pd.read_csv('data/vn_sentiment/train.csv')

# replace new lines with space
data['comment'] = data['comment'].str.replace('\n', ' ')

data.head()

,id,comment,label
0,train_000000,Dung dc sp tot cam on shop Đóng gói sản phẩm ...,0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng...,0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp ...,0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọ...,1
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đ...,1


In [3]:
# from sent_to_vec.masked_lm.bert_model import BertLMWrapper
# from sent_to_vec.masked_lm.train import LanguageModelLearner
# from sent_to_vec.masked_lm.vi_data import ViTextDataset

# from common.modules import BertAdam
# from common.callbacks import PrintLoggerCallback, EarlyStoppingCallback, ModelCheckpointCallback, TensorboardCallback, ReduceLROnPlateau

In [4]:
# model = BertLMWrapper(from_fp='bert_vi_base.bin')
# model.init_model()

# dataset = ViTextDataset()
# dataset.initialize(model, data_texts=list(data['comment']))

In [5]:
# BATCH_SIZE = 16
# n_epochs=10

# learner = LanguageModelLearner(model,
#     optimizer_fn=BertAdam,
#     optimizer_kwargs={
#         'lr': 1e-5,
#         't_total': n_epochs * (len(dataset) // BATCH_SIZE),
#         'warmup': 0.04
#     })

In [6]:
# learner.fit(
#     training_data=dataset,
#     batch_size=BATCH_SIZE,
#     epochs=n_epochs,
#     callbacks=[
#         PrintLoggerCallback(log_every=1, metrics=['loss']),
#     ],
#     gradient_accumulation_steps=10,
#     clip_grad=1.0,
#     fp16=True
# )


In [7]:
from text_classification.with_pretrained.model import LMClassifierWrapper
from text_classification.with_pretrained.train import LMClassifierLearner

Default language for this instance: en


In [8]:
# classifier = LMClassifierWrapper({
#     'encoder': model
# })
# classifier_learner = LMClassifierLearner(
#     classifier,
#     optimizer_fn='adam'
# )

In [9]:
# classifier_learner.fit(
#     training_data=(data['comment'], data['label']), 
#     batch_size=BATCH_SIZE,
#     epochs=50, 
#     callbacks=[PrintLoggerCallback(log_every=5)]
# )

In [10]:
test_data = pd.read_csv('data/vn_sentiment/test.csv')

# replace new lines with space
test_data['comment'] = test_data['comment'].str.replace('\n', ' ')

In [11]:
test_data.head()

,id,comment
0,test_000000,Chưa dùng thử nên chưa biết
1,test_000001,Không đáng tiềnVì ngay đợt sale nên mới mua n...
2,test_000002,Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắc...
3,test_000003,Vải đẹp.phom oki luôn.quá ưng
4,test_000004,Chuẩn hàng đóng gói đẹp


In [21]:
import torch
test_item = [test_data['comment'][2]]

In [22]:
classifier = LMClassifierWrapper(from_fp='bert_vi_sentiment.bin')
classifier.init_model()
classifier(test_item, return_logits=True)[0]

{'num_words': 30000, 'hidden_size': 512, 'num_hidden_layers': 6, 'num_attention_heads': 8, 'intermediate_size': 1140, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'attention_probs_dropout_prob': 0.1, 'max_position_embeddings': 100, 'featurizer_seq_len': 100, 'type_vocab_size': 2, 'initializer_range': 0.02, 'use_adasoft': True, 'append_sos_eos': True, 'featurizer_reserved_tokens': ['<START>', '<STOP>', '<UNK>', '<MASK>'], 'tokenize_fn': <function word_tokenize at 0x7f926b9c3598>, 'input_shape': (None,), 'adasoft_cutoffs': [2000, 4000, 10000]}


tensor([[0.9972, 0.0028, 0.0000]])

In [27]:
classifier([test_data['comment'][0]])

[[{'intent': 0, 'confidence': 0.999737560749054},
  {'intent': '<unknown>', 'confidence': 0.00020573583606164902}]]

In [15]:
raw_model = classifier.model

In [16]:
classifier.label_encoder.classes_.tolist().index(1)

1

In [17]:
_, seq_tokens = raw_model.encoder(test_item)
sequence_output = raw_model.rnn(seq_tokens)[0]
print(sequence_output)

output, idxs = torch.max(sequence_output, 0)
print(idxs)
idxs = idxs.data.cpu().numpy()

sent = raw_model.encoder.featurizer.transform(test_item)
raw_sent = raw_model.encoder.featurizer.inverse_transform(sent)
print(raw_sent)

import matplotlib.pyplot as plt

argmaxs = [np.sum((idxs == k)) for k in range(len(sent[0]))]
# argmaxs[0] = 1e-8
print(argmaxs)
x = range(len(sent[0]))
y = [100.0 * n / np.sum(argmaxs) for n in argmaxs]
print(y)

plt.xticks(x, raw_sent[0], rotation=45)
plt.bar(x, y)
plt.ylabel('%')
plt.title('Visualisation of words importance')
plt.show()

TypeError: ones_like() received an invalid combination of arguments - got (list), but expected one of:
 * (Tensor input, torch.dtype dtype, torch.layout layout, torch.device device, bool requires_grad)
 * (Tensor input, bool requires_grad)
